> Не забываем в CLI переключаться в нужный environment!

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
import mlflow
from mlflow.models.signature import infer_signature

In [ ]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("ML_Experiment")

## MLFlow examples

> Неодопустимо перезаписывание param в рамках одного run

In [ ]:
# Логирование 1 параметра эксперимента
train_size = 0.8
mlflow.log_param("train_size", train_size)

In [ ]:
# Логирование сразу нескольких параметров
penalty = "l2"
fit_intercept = False

mlflow.log_params({
    "penalty": penalty,
    "fit_intercept": fit_intercept,
})

In [ ]:
lr = LogisticRegression(
    penalty=penalty,
    fit_intercept=fit_intercept,
)

In [ ]:
X,y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=train_size)

In [ ]:
# Logging datasets

mlflow.log_input(
    mlflow.data.from_numpy(X_train, name="X_train"),
    context="train"
)
mlflow.log_input(
    mlflow.data.from_numpy(y_train, name="y_train"),
    context="train"
)

In [ ]:
# Or save localy first
pd.DataFrame(X_train).to_parquet("X_train_1.parquet")
mlflow.log_artifact(local_path="X_train_1.parquet", artifact_path="X_train_1")

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
pred = lr.predict(X_train)

In [ ]:
mlflow.log_input(
    mlflow.data.from_numpy(pred, name="pred"),
    context="train_predict"
)

In [ ]:
accuracy_train = (pred == y_train).mean()
accuracy_test = (lr.predict(X_test) == y_test).mean()

In [ ]:
mlflow.log_metrics({
    "accuracy_train":accuracy_train,
    "accuracy_test": accuracy_test,
})

In [ ]:
mlflow.log_input(
    mlflow.data.from_numpy(X_test, name="X_test"),
    context="test"
)
mlflow.log_input(
    mlflow.data.from_numpy(y_test, name="y_test"),
    context="test"
)

In [ ]:
input_example = X_test[:1]
signature = infer_signature(input_example, lr.predict(input_example))

mlflow.sklearn.log_model(
    sk_model=lr,
    artifact_path="Logistic Regression v1",
    signature=signature,
    input_example=input_example
)

## Previous Runs

In [ ]:
mlflow.search_runs()

In [ ]:
mlflow.last_active_run().info

In [ ]:
mlflow.active_run().info

## Load Artefacts From MLFlow

### Model

In [ ]:
# Берём из сервиса или из run

# ../experiment_id/run_id/artifacts
uri = mlflow.last_active_run().info.artifact_uri
uri += "/Logistic Regression v1"

lr_load = mlflow.sklearn.load_model(
    model_uri=uri
)

In [ ]:
lr_load.coef_

In [ ]:
lr.coef_

### Dataset

> Если сохраняли как артефакт, то можно вовсе через ui скачать

In [ ]:
uri = mlflow.last_active_run().info.artifact_uri

In [ ]:
# Как артефакт

artifact = mlflow.artifacts.download_artifacts(f"{uri}/X_train_1/X_train_1.parquet")
pd.read_parquet(artifact)

In [ ]:
# Как mlflow.data.dataset - тут я не понял как скачивать!

run = mlflow.get_run(mlflow.last_active_run().info.run_id)

for i, d in enumerate(run.inputs.dataset_inputs):
    print(i, d.dataset.name)

In [ ]:
# dataset_info = run.inputs.dataset_inputs[0].dataset
# d = mlflow.data.get_source(dataset_info)
# d.load()

In [ ]:
mlflow.end_run()